In [ ]:
from aopy import datareader, datafilter
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn import model_selection
from scipy import signal, stats
from numba import cuda
import time

# Functions
def IMA_preprocess(data_in, time_start, time_stop, srate, on, ftype, order, Wn, num_ch, verbose = True, var_thresh = False):
    '''
    This function preprocesses data before dimensionality reduction
    '''
    
    # Crop to a specific time range
    if type(time_start) != int or type(time_stop) != int:
        stidx = (time_start*srate).astype(int)
        spidx = (time_stop*srate).astype(int)
    else:
        stidx = time_start*srate
        spidx = time_stop*srate
    
    # Check that data doesn't contain any artifacts if set to a max variance value
    if type(var_thresh) is int:
        data_in_var = np.var(data_in[:,stidx:spidx])
        if data_in_var > var_thresh:
            return None
    if verbose == True:
        print('Start Index: ' + str(stidx))
        print('Stop Index: ' + str(spidx))
    data = data_in[:,stidx:spidx]
    
    # Butterworth filter
    if on is True:
        sos = signal.butter(order, Wn, ftype, fs = srate, output = 'sos')
 
        data_out = signal.sosfiltfilt(sos, data, 1)    
    
    #print(data_out.shape)
    
    return data_out


def IMA_pca(data_in, ncomp_pca):
    '''
    This function performs principal componant analysis (PCA) on input data
    '''
    zdata = stats.zscore(data_in, 0)

    pca = PCA(n_components = ncomp_pca)
    
    return pca.fit(zdata)


In [ ]:
'''
Plot for SVD testing
'''
srate = data_param['srate']
# Plot raw data and unfiltered signals
chanidx = 1
timeaxis = np.arange(0, proc_param['time'], 1/srate)
fig0, ax0 = plt.subplots(1)        
ax0.plot(timeaxis, data_in[chanidx,0:(srate*proc_param['time'])], label = 'Raw')
ax0.plot(timeaxis, data_proc[chanidx,0:(srate*proc_param['time'])]     , label = 'Filtered')
plt.xlabel('Time [s]')
plt.xlim(0, 1)
ax0.legend(prop = {'size': 16})
fig0.set_size_inches(20,5)

# Freq domain: data and filtered data
fDelta = 1/np.shape(data_proc)[1]*srate
fftAxis = np.arange(0, srate, fDelta)
fig1, ax1 = plt.subplots(1)
ax1.plot(fftAxis, np.absolute(np.fft.fft(data_in[chanidx, 0:(srate*proc_param['time'])])), label = 'Raw')
ax1.plot(fftAxis, np.absolute(np.fft.fft(data_proc[chanidx, 0:(srate*proc_param['time'])])), label = 'Filtered')
plt.xlabel('Frequency [Hz]')
plt.xlim(0,100)
ax1.legend(prop = {'size': 16})
fig1.set_size_inches(20, 5)

# PCA Cumulative Variance
fig2, ax2 = plt.subplots(1)
ax2.plot(np.cumsum(pca_out.explained_variance_ratio_)*100)
plt.xlabel("Latent Dimension")
plt.ylabel("Percent Explained Variance")
plt.title("Cumlative Variance")
plt.show()

In [ ]:
srate_in= data_param['srate']
num_ch = data_param['num_ch']
# plot the data 
T = 20 # (s)
n_plot = srate_in*T
plot_idx = np.arange(n_plot)
ch_idx = 10
time = plot_idx/srate_in
f,ax = plt.subplots(1,1,figsize=(20,8))
ax.plot(time,data_in[ch_idx,plot_idx])
np.shape(data_in[ch_idx,plot_idx])
ax.set_xlabel('time (s)')
ax.set_ylabel('($\mu$V)')
ax.set_title('ECoG ch. {}'.format(ch_idx+1))